# MobileNet Tumor Classification

## Libraries

In [ ]:
pip install numpy matplotlib tqdm torch torchvision torchinfo seaborn scikit-learn opendatasets

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

import torch
import torchvision
from torchinfo import summary
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

## Dataset

To download the dataset:
1. Go to 'https://www.kaggle.com/settings' and under 'API' click on create new token. A json file will be downloaded. 
2. Place that file into the same folder of the current script.

In [ ]:
# Downloading the dataset from Kaggle and save it under the current folder ('/.')
import opendatasets as od

od.download("https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset")
# Dataset saved in ./brain-tumor-mri-dataset

## Execution Control Panel

In [ ]:
# Model
model_name = 'MobileNet'
FREEZE_FEATURE_EXTRACTOR = False   # To freeze the base layers (features section)

# Paths (if we download the dataset)
TRAIN_DIR = "./brain-tumor-mri-dataset/Training"
TEST_DIR = "./brain-tumor-mri-dataset/Testing"

# Paths (on Kaggle)
# TRAIN_DIR = "/kaggle/input/brain-tumor-mri-dataset/Training"
# TEST_DIR = "/kaggle/input/brain-tumor-mri-dataset/Testing"

# Dataset Parameters
CATEGORIES = ["glioma","meningioma","notumor","pituitary"]
IMG_SIZE = 224

# ============= Check if CUDA is available =============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============= PRE-TRAINED MODEL UNDER EVALUATION =============
model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1').to(device)

# ============= Training Model Parameters =============
BATCH_SIZE = 32
EPOCHS = 10

## Model Creation

### Working on the *Classifier*
Adjusting the output layer or the classifier portion of our pretrained model to our needs.

In [5]:
output_shape = len(CATEGORIES)  # Get the length of class_names

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)

### Working on the *Feature Extractor*
For parameters with requires_grad=False, PyTorch doesn't track gradient updates and in turn, these parameters 
won't be changed by our optimizer during training

In [6]:
# Doing the freezing
if FREEZE_FEATURE_EXTRACTOR:
    for param in model.features.parameters():
        param.requires_grad = False

### Final Model

In [ ]:
# Printing the Final Model
summary(model=model, 
        input_size=(32, 3, IMG_SIZE, IMG_SIZE),
        col_names=["num_params", "trainable"],
        col_width=15,
        row_settings=["var_names"]
)

## Train and Test Sets

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.95, 1.05)),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
test_dataset_clean = datasets.ImageFolder(TEST_DIR, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_clean = DataLoader(test_dataset_clean, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## Training the CNN (*on the GPU*)

In [9]:
criterion = nn.CrossEntropyLoss()  # loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training function with progress bar
def train_epoch(model, dataloader, criterion, optimizer, device, epoch_num, total_epochs):
    # Set model to training mode - enables dropout, batch norm updates, etc.
    model.train()
    
    # Initialize tracking variables for this epoch
    running_loss = 0.0  # Accumulate loss across batches
    correct = 0         # Count correct predictions
    total = 0           # Count total samples processed
    
    # Create progress bar for training batches
    train_bar = tqdm(dataloader, desc=f'Training Epoch {epoch_num}/{total_epochs}', leave=False, dynamic_ncols=True)
    
    # Iterate through batches in the training dataset
    for batch_idx, (data, target) in enumerate(train_bar):
        # Move data and labels to GPU/CPU device
        data, target = data.to(device), target.to(device)
        
        # Clear gradients from previous iteration (PyTorch accumulates gradients by default)
        optimizer.zero_grad()
        
        # Forward pass: compute model predictions
        output = model(data)
        
        # Calculate loss between predictions and true labels
        loss = criterion(output, target)
        
        # Backward pass: compute gradients via backpropagation
        loss.backward()
        
        # Update model parameters using computed gradients
        optimizer.step()
        
        # Track metrics for this batch
        running_loss += loss.item()  # .item() extracts scalar value from tensor
        
        # Get predicted class (index of maximum logit)
        _, predicted = torch.max(output.data, 1)  # Returns (max_values, indices)
        
        # Update counters
        total += target.size(0)  # Add batch size to total count
        correct += (predicted == target).sum().item()  # Count correct predictions
        
        # Update progress bar with current metrics
        current_acc = 100. * correct / total
        current_loss = running_loss / (batch_idx + 1)  # Average loss so far
        train_bar.set_postfix({
            'Loss': f'{current_loss:.4f}',
            'Acc': f'{current_acc:.2f}%'
        })
    
    # Calculate final epoch metrics
    epoch_loss = running_loss / len(dataloader)  # Average loss across all batches
    epoch_acc = 100. * correct / total          # Final accuracy percentage
    return epoch_loss, epoch_acc

In [ ]:
# Training loop
start_time = time.time()

# Training loop
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    
    # Training phase
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch + 1, EPOCHS)
    
    # Calculate time for this epoch
    epoch_time = time.time() - epoch_start_time

# Final results
end_time = time.time()
full_finetune_times = end_time - start_time

# Save final model
model_name_ext = model_name + '.pth'
torch.save(model.state_dict(), model_name_ext)

print("TRAINING COMPLETED!")
print(f"Total training time: {full_finetune_times:.2f} seconds ({full_finetune_times/60:.2f} minutes)")
print(f"Average time per epoch: {full_finetune_times/EPOCHS:.2f} seconds")

## Evaluating the Model (*on the CPU!*)

In the following snippet of code the '.cpu()' call moves a PyTorch tensor from the GPU back to the CPU.

PyTorch tensors must be on the CPU to be converted into NumPy arrays using .numpy(), because .numpy() only works on CPU tensors.

In [ ]:
model.eval()

running_loss = 0.0
correct = 0
total = 0

all_preds = []
all_targets = []

with torch.no_grad():
    for data, target in test_loader_clean:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        running_loss += loss.item()
        
        # Get predictions
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        
        # Collect confidence scores
        probs = torch.softmax(output, dim=1)
        max_probs = torch.max(probs, dim=1)[0]

# Compute test loss and accuracy
clean_test_loss = running_loss / len(test_loader_clean)
clean_test_acc = 100. * correct / total
print(f'Test Loss: {clean_test_loss:.4f}, Test Acc: {clean_test_acc:.4f}%')

In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',xticklabels=CATEGORIES, yticklabels=CATEGORIES, annot_kws={'size': 17})

plt.xlabel('Predicted Label', fontweight='bold', fontsize=20)
plt.ylabel('True Label', fontweight='bold', fontsize=20)
plt.xticks(fontsize=15) 
plt.yticks(fontsize=15)
plt.tight_layout()

plt.show()

In [ ]:
# Classification Report
print("\nTest - Classification Report:")
report = classification_report(all_targets, all_preds, target_names=CATEGORIES)
print(report)